In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.genz.file_names import project, Optsd
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Optsd()
style = {'description_width': 'initial'}
input_selkey = widgets.Dropdown(options=sorted(get_h5_keys(opts.info_fname, key='auto_qc')), disabled=False,
    description='Subject DWI to qc:', style=style)
input_selkey.layout.width = '350px'
input_selkey

/home/toddr/Software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


Dropdown(description=u'Subject DWI to qc:', layout=Layout(width=u'350px'), options=('/sub-genz103/ses-1/dwi/auto_qc', '/sub-genz104/ses-1/dwi/auto_qc', '/sub-genz201/ses-1/dwi/auto_qc', '/sub-genz203/ses-1/dwi/auto_qc', '/sub-genz205/ses-1/dwi/auto_qc', '/sub-genz301/ses-1/dwi/auto_qc', '/sub-genz302/ses-1/dwi/auto_qc', '/sub-genz303/ses-1/dwi/auto_qc', '/sub-genz305/ses-1/dwi/auto_qc', '/sub-genz308/ses-1/dwi/auto_qc', '/sub-genz501/ses-1/dwi/auto_qc', '/sub-genz502/ses-1/dwi/auto_qc', '/sub-genz503/ses-1/dwi/auto_qc', '/sub-genz506/ses-1/dwi/auto_qc', '/sub-genz508/ses-1/dwi/auto_qc', '/sub-genz510/ses-1/dwi/auto_qc', '/sub-genz906/ses-1/dwi/auto_qc'), style=DescriptionStyle(description_width=u'initial'), value='/sub-genz103/ses-1/dwi/auto_qc')

In [2]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
# new h52df function makes all cols numeric. yes!
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
# todo: make python scriptlet to load files into FSLeyes
# fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
# with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
#     with open('fslview_cmd.sh', 'w') as f:
#         f.write('fslview ' + fslv_names + ' &\n')

# limit to only active rows with mask
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] != -9999, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] != -9999, 'itopdn_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] == -9999, 'itopup_visqc'] = False
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] == -9999, 'itopdn_visqc'] = False
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)

In [3]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600),]
#items_layout = [widgets.Layout(flex-basis='50%', ), widgets.Layout(flex-basis='50%', widgets.Layout(flex-basis='100%',)]
box_layout = widgets.Layout(display='flex', align_items='stretch', border='solid', width='100%')
top_box = widgets.HBox(children=items)
bottom_box = widgets.HBox(children=[dwi_w])
dwi_box = widgets.Box([top_box, bottom_box], layout=box_layout)
dwi_box

Box(children=(HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xbf\xbf\xbf\x1f\x1f\x1f___???\xdf\xdf\xdf\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xf5Uk\xc9\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\x9d\xe9\x96\xa2\xba\x1a@\xaf\x0cy9~\xd6l\xcd}\x16\xbc\xff#\\3\x92\t\x15\x0b\xd1\xe8\xdeku\x97\x85H\x02E\xb6_B\x86\xff\r\x00\x00\x85\xf0\xbfKg\x00\x00\xe0X\x10\x16\x00\x14\x03\xc2\x02\x80b@X\x00P\x0c\x08\x0b\x00\x8a\x01a\x01@1 ,\x00(\x06\x84\x05\x00\xc5p7\xc2\xea\xfb\xfa\xd2Y\xb8\x10\xbd\xa5\xae\x9bJo\xea\xfa\x98f\xb7\xb5\xee\xfb\xdc\xe77M]\xcb\xcfn\xa2\xedU\xd7\xee\xb6\xb7\xc9\xf6\x0c\x13G^\x87z\xbc\x00qf7}t[d\xae\xd5\xdc\xb4\x0e\xec!\x9a\xfa\xe8;\xf1~\xef\xd9i\x10\xd6\x92l\x9a]\x19\x0eKE%o\xd06\xdc\xd2\xca\xa2sJy8\x80\xc8\x16\x97PLBn\x9a!\xac\xce+\xef\x9d\x7f\x16\xed\xb8\xfd\xa0\xb2\xce&,{\xc6\xd1\xd9\xc4\x89{\xb4\xc2{K\x9d\x83\xff\x87\xc8\\\xabY\x1c<Q\x91\xe6o\x9a\xec=\xbb\xd1_\x13b\xdc\x92\xbb\xc5\x0eo)\x14\x84\xb5\x1c\xa2MKE\x1bo\xc9\xed\xb4\x14\x9b)a5\n\x95r-S\xdd\xe8\r\x8d{K96S\xdatnw\xe5\xa3\xad\xc3,o\x82\xed]\xfc\xb9\x88\xb3\tks\x9c\xb0\xccI\xaa\xccz\xb7\x81p\xae\xb6d\xae\xd5,\x0e\x9e\xe8\xee\x92\xb7\xdd\xb1\xde\xc8\xdc\xb3\x95\xd3\xaf\xbb\xe6\xc9-v\xd4\x96RAX\xcb!\xef\x8a\xb6\xeb\x1a\xefV\x97\x9b\x1a\xbdEo\x10\xb5\xde\xa2\xf6]8}\x91/.\xde\x99o\xea0\xd5\xf0\x03\xe9\xc7Un;\xa1_w\xdeyU2\xfb:4Q\xa1\xd6\x81p\xf1\\\xc2rg\xdcz\xc4\x7fi?q\xe9\xd9\x8d\xff[\x13\xee\xbd\xe7Z\x1d\xc5\xc1\x13\xdd\xed \x8e>Zz\xcfV\xbd\xbe}\xeaQ\xb4\xc9-v\xd4\x96bAX\x8b\xb1\xbb\xfdkUpeX\xd2\xb9M"\xd8\xd2\xd8o9Y\xd6\x0e\x05&3\xd3\xaf\xe3\xd8B\xe3\x9fy\x94\xea!a\xb5A\x94!\xc6"\x1c\x1c\xa69X\xb2\xcf$\xac\xec\x19ob}\x06\x89o\xfc\x88jwzR\xb7\xde\xee{\xae\xd5Q\x1c#\xac\xe3\x8f\x96\xde\xb3\xad\xbd}6\xf6["\xbd\xc5\x8e\xd9R.\x08k1\x1aw\xeb\x0b\x9bXm7\t[\xb2\xc6o\xd8j\xd1\x10\xab\xa93\x95!Mp\xe6U\xf0\xdb\x01am\xa2Z\x91,\xc2*>\xe9\x92\x8a\x94\xbf[\xca9\x845q\xc6")\x93A\xe2\xc2\xbb\xea\xea\xcf\x14\x18l\xdf\xb5:\x8a3\x0b\xcb\xcb\x91\xfd\x13do\xb1\x83[\xca\xe5v\x84e\x1a#\x8d3\xbc\xbf\xb5~)\xff\x17\xb2\x05\xb95U\x9c\x88N~yu\xfe\x1d\xa5\x0f\xe8*\x10\xea\r\xddv\xe9\xd2\x08\x9aM\xda\xf1f0\x07\x11AP\xbe\t\xf3%\x12a\x85g\x90\xe6 \xb3\x87\xa3\xee\x8f\x13\xd6\x10D\x14\x07\x84U\xc7\xc1\x8a-0Q\xc5\xa6\xf1\xebY\xb9\x9c\xaa#\xebM\xf1\xd9\xb9\xc0S\xbf\xb1q\xf2\xeb\x8cy\xa6\xcey\xe2\x8c\xdb\xe4k`RX:\x85\xe0\x10{\xae\xd5\x98#\xff\x94\xfd\x93\xf0\xd3\xca\xe7\xba\xf7\xf3\x1c~T_\xa2:\xb8\x90\xc9=\xeb\xe5\xc0\xdc[\xe9-v\xcc\x96\x00\x93\xc2\x9e[\xff\x9a\xb8\x15a\x8d\x8d\x91\xfa\xfb2\',\xbbO\x9d\x16x\xf7\x96k\x15\x11\xf6\x806\xc6\x90otQ\x1a\xa1\xb0Z\xf7Um\x9e\\o\xc6\xfb\xcb\xbe\x1c\x1d\x10~\xb5\xa7g\x90\xe6 \xd9\xc3\xff\xf4f\xc7\xc16\xac0O\x87\x84\x95\x89\x01u\x11\xae\xa2\x1c\x88\xaa\x12\xe1\'\xa3\x9c\xca#\xb7SgW\xf9\xb9\x92\xed,\x1b?\xa9\xc9s\xce\x9f\xf1&\r\xf6\x82}:\xaf\xc4j\xed\x06\xb6\xdds\xad\x82\x82o4\x17\x9e\xc4\x98\xd6T\xae=a\xc5\x1f\xb5\xb7W$\xac\x

In [4]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xdf\xdf\xdf___???\xbf\xbf\xbf\x1f\x1f\x1f\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xa8*\xe1\x90\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\xdd\t\x96\xa2X\xb7\x06\xd0Gw\'\xc70\xfe\x85\xf3\x1f\xc2\x93\xe6"\x9d\x86d\x1aD\x9d\xc8\xbd\xd7\xaa\xca\x10\x81\xc3E\xf8\xa4\xf7\xffn\x00A\xfc\xdfOO\x00\xc0\xbb\x04\x16\x10\x86\xc0\x02\xc2\x10X@\x18\x02\x0b\x08C`\x01a\x08, \x0c\x81\x05\x84\xf1{\x03\xab\xeb\xd2OO\xc2\x0f\xe9\xb2\x94\xaaf\xec\xd4v[\xd5\xbdk\xea\xba\xa3\xe1\x8b*\xa5~\xd8b\xd3\xbdi\xeb{\xf7z\xd7\xfd\xc0\x931\x7f\x93\xf4h\xf1v\xea\x8an\xb3\x1c\x1c\xcc\x9c\xb3\xb5\xbe\xe8\xa3\xac\xd2\xba\xe4\x8f\xfam\xab\x81\xc0\xfa+Eu_\x87\xd7+I\xd3/\xaf\xf5\xbaK\xdd\xafI\x7f\xb2z|\xa1<\\{\xd6\xc1T\xf6\x9dN\x04V\xbbX\xfd\xdbe+\xeaG\xf7/#\xeb{\x02k?k\xe7j\x0bu\xb9xk\x98\xe8\xe5\x9c?\x989\xa7|\xd9\xb2r\x18\xf3\x9bck\xab\xea\xf4\x14|a\xb3\xb4\x1d\xae\x06\xcffd\x00\x02\xeb/\x94\xf5~%\xa9\xb7]\x8ez\xfa\x94\xe2Y`U\x83\xa1r\xea\xab\x16c\x87j~k\xc8\xd8\x83\x95o\x9c\xda\xfb\x02_\xa7\xf5$\x17\xab\xee\xedv\xb8\x8do\t\xac\xdd\xac]T\x9bZ5L]z\xbcS\xce\xe1\x9c\x1d\xcc\x9cS\xbel\xd9}\x1e\xd7\xed\xbbY\xf0\xf1\xf9\xb4[\xda\x8eV\x83\xa732\x00\x81\xf5\x17\xfa\x0f\xben\xdbj\xb1\xe4\xf7\x9d\xaa\xb1\xcb\xd8\xa1Lc\x97\xa1\xdf\x0f\xd7/\x8f\x97\xf7E\xcb\x8b\xb4\xae\xba\x1e`?\xf80\xb5\xed\xd8\x96\xb2]\xb4\xab\xe9\'\x7f\xfc\xd6\x1e6\xb5\xbe\xd8\\\xfc\x8e\xc0\xda\xcd\xda\xc3j}\xb0\x16\xcbW\xd5\xba\xff\x173\xe7-_\xb6\xec\xde\xc3\xfb9\xf0\xe9\xf9\xb4_\xda\x0eV\x83\xe732\x00\x81\xf5\xe7\xeekC\x1aV\xdc\xfek\xad\x9d;\x95\xab.U\xfe"\xeb\x97\xa5\xaf6LN\xd6O\xc7\xfb\x1e\xcb\x96o\xaa~\x15X\xf5j\xa3\xa3|\xac\xd1\xab\xd1T_\xae\xe8\xdf\x10X\xfbY{\\\xadXnQ\xdd\xfb\xed\xf3u\x11\xaf/f\xce[\xde\t\xac\x0f\x8e\xed\xa4\xfd\xd2\xb6_\r^\xcc\xc8\x00\x04\xd6\x9f\xab\xe65\xa1\xcc\xc5R\xeeT\xe6,y|\xe16\x1f\xdd\xc4\xaa\xa6\x037\x07o\xadZ\xde\xac^}\x11XE\xb7\xdeI\xea\x17\xfbas\xa5\xdd\xedW\xbd\xde\x8a\xf8\x86\xc0\xda\xcf\xda\xe3j\xe5b6\x0f\x9f\xcb*\xc1^\xcd\x9cw\'\xe3/{\xf8\xe3\x9e\xdf\x1b\xdffi;\xdc\x1e}6#\x03\x08\x1cX\xc5x\xcej\xca\x8c\xc5\x073\xfe\xd9\xff\xbf\xec\x8f \xd7\xed\xe1\xca\xd5\xd6\xfd[\xcbef\x1c\xe1\xbc?1\xbc1\x1e\x9e\x9ck,\x0c_\xdf\xab~\x17\xc15\xbcY\xac\xa7\xab\xdc\x05\xd6\xba\x05\xfb)8\xe8c\x96\xba\xf7\x02\xeb\xb6\xda\xc0\xf8"\xb0\xd2jc\xe4\xf6X\xa37\xfb9\xd5r\xb7\xebhJ\x871\x8f\x9d\xb6\xad\x9b7<\xc77\x8a9\xfc\xda\xe9\x0b\xff\xb8\xcd\x07\xb3\xf6\xb8\x1d\xcb\xd9<\x8er5\x97^\xcc\x9c\xc7$,\xdb\xb8\x9c\xeae\xad\xe3\xc9\xec\x16\x1f\xcb\xd1\x90\xf7\xad\xe2\xe2\xb0\xe7m\xef\xc3t\x8e\xcbh\xb9\x9b\xf2\xe9\xcfW\xfd\x94\x8b\xc0Z\xad\x06\xaff\xe4\xf0\xfe\xd8\xf3\x8b\x15\xe3G\x85\r\xacf>54~}\x1e\x05V\xee\'\xedW\xf8\xf9\xad\xf98P\x99G\x98\xb71\xfa7\xdaM\x8du`\xd5\xf37w\x1a\xab\x17\x8f\x05>\xff\xf9\xc8\x80\xf57\xfd\xbe\x05\xfb)\xd8\xf5\xb1\x1c\xba\xb8\xfb\xf2\x18\xd6z\x9a\xbe\n\xac\x83m\xc0q\x8dn6SP6\xcd\xfaK`;\xa5\xfd\x

In [5]:
dwi_w.get_changed_df()
topupdn_w.get_changed_df()
all_dwi_visqc = pd.concat([dwi_w.get_changed_df(), topupdn_w.get_changed_df()], axis=1)
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))

In [5]:
topupdn_w.get_changed_df()

,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc
0,0,0,0,True,0,0,True
1,1,1,0,True,1,0,True
2,2,2,0,True,2,0,True
3,3,3,0,True,3,0,True
4,4,4,0,True,4,0,True
5,5,5,0,True,5,0,True
6,6,6,0,True,6,0,True
7,-9999,7,0,False,-9999,-9999,False
8,-9999,-9999,-9999,False,-9999,-9999,False
9,-9999,-9999,-9999,False,-9999,-9999,False


In [7]:
dwi_w.get_changed_df()

,bvals,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec
0,0,0,0,True,0.000,0.000,0.000
1,2000,0,1,True,1.000,0.000,0.000
2,2000,1,0,True,0.000,1.000,0.000
3,2000,2,0,True,0.000,0.000,1.000
4,2000,3,0,True,-0.625,-0.777,0.076
5,2000,4,0,True,-0.745,0.623,0.237
6,2000,5,1,True,0.025,-0.095,0.995
7,2000,6,0,True,-0.853,-0.004,-0.522
8,2000,7,1,True,-0.155,0.904,-0.399
9,2000,8,1,True,-0.025,-0.716,-0.698
